### Ray AI Libraries

In [3]:
import asyncio
import fastapi
import pandas as pd
import requests
import xgboost
from pydantic import BaseModel
from sklearn.model_selection import train_test_split

import ray
import ray.tune
import ray.train
from ray.train.xgboost import XGBoostTrainer as RayTrainXGBoostTrainer
from ray.train import ScalingConfig, RunConfig
import ray.data
import ray.serve


### 3 Step -> 0 : Cloud, 1: Ray Core (low level distr. computing framework), 2: Ray AI Libraries (Data, Train, Tune , Serve)